In [1]:
import random
import math

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    get_linear_schedule_with_warmup,
)

# TraceML imports
from traceml.decorator import trace_model_instance
from traceml.manager.tracker_manager import TrackerManager

In [3]:
SEED = 42
MODEL_NAME = "distilbert-base-uncased"
MAX_TRAIN_EXAMPLES = 500
MAX_VAL_EXAMPLES = 100
BATCH_SIZE = 32
EPOCHS = 1
LR = 2e-5
WARMUP_RATIO = 0.06


In [4]:
def set_seed(seed: int = SEED):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def accuracy_from_logits(logits: torch.Tensor, labels: torch.Tensor) -> float:
    preds = torch.argmax(logits, dim=-1)
    correct = (preds == labels).sum().item()
    return correct / max(1, labels.size(0))


def prepare_data():
    raw = load_dataset("ag_news")
    train_raw = raw["train"].select(range(min(MAX_TRAIN_EXAMPLES, len(raw["train"]))))
    val_raw = raw["test"].select(range(min(MAX_VAL_EXAMPLES, len(raw["test"]))))

    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    def tok(examples):
        return tokenizer(examples["text"], truncation=True)

    train_ds = train_raw.map(tok, batched=True, remove_columns=["text"])
    val_ds = val_raw.map(tok, batched=True, remove_columns=["text"])

    train_ds = train_ds.rename_column("label", "labels")
    val_ds = val_ds.rename_column("label", "labels")

    collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

    train_loader = DataLoader(
        train_ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collator
    )
    val_loader = DataLoader(
        val_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collator
    )
    return tokenizer, train_loader, val_loader


In [5]:
set_seed()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if torch.cuda.is_available() else torch.float32

tokenizer, train_loader, val_loader = prepare_data()

model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=4
).to(device)

# Attach TraceML hooks
trace_model_instance(
    model,
    sample_layer_memory=True,
    trace_activations=True,
    trace_gradients=True,
)

optimizer = AdamW(model.parameters(), lr=LR)
total_steps = EPOCHS * math.ceil(len(train_loader))
warmup_steps = int(WARMUP_RATIO * total_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/xn/64_d8wlj1sgbfl7wf19vt0b40000gn/T/ipykernel_86974/2174213543.py:26: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())


In [6]:
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    scaler,
    device,
    dtype,
    epochs: int = 1
):
    """
    Train and validate a model.

    Args:
        model: torch.nn.Module
        train_loader: DataLoader for training
        val_loader: DataLoader for validation
        optimizer: torch optimizer
        scheduler: learning rate scheduler
        scaler: GradScaler for mixed precision
        device: torch.device ("cpu" or "cuda")
        dtype: torch dtype (torch.float16 or torch.float32)
        epochs: number of training epochs
    """
    model.train()
    global_step = 0

    for epoch in range(epochs):
        running_loss = 0.0
        running_acc = 0.0

        for batch in train_loader:
            batch = {k: v.to(device) for k, v in batch.items()}

            optimizer.zero_grad(set_to_none=True)
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=dtype):
                out = model(**batch)
                loss = out.loss
                logits = out.logits

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()

            acc = accuracy_from_logits(logits.detach(), batch["labels"])
            running_loss += loss.item()
            running_acc += acc
            global_step += 1

        # Validation
        model.eval()
        val_loss, val_acc, n_batches = 0.0, 0.0, 0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=dtype):
                    out = model(**batch)
                    loss = out.loss
                    logits = out.logits
                val_loss += loss.item()
                val_acc += accuracy_from_logits(logits, batch["labels"])
                n_batches += 1

        model.train()

In [7]:
tracker = TrackerManager(interval_sec=1.0, mode='notebook')
tracker.start()
train_model(model, train_loader, val_loader, optimizer, scheduler, scaler, device, dtype)
tracker.stop()
tracker.log_summaries()


[12:55:22] ERROR traceml.SystemSampler: [TraceML] WARNING: GPU not available: NVML Shared Library Not Found
/var/folders/xn/64_d8wlj1sgbfl7wf19vt0b40000gn/T/ipykernel_86974/4035887814.py:42: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=dtype):


VBox()

/var/folders/xn/64_d8wlj1sgbfl7wf19vt0b40000gn/T/ipykernel_86974/4035887814.py:47: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
  scaler.scale(loss).backward()
/var/folders/xn/64_d8wlj1sgbfl7wf19vt0b40000gn/T/ipykernel_86974/4035887814.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=torch.cuda.is_available(), dtype=dtype):


[Val] epoch 1 | loss 1.4251 | acc 0.2891


╭─────────────────────────────────────────────── System - Summary ────────────────────────────────────────────────╮
│ TOTAL SYSTEM SAMPLES |                       117                                                                │
│ CPU AVERAGE          |         97.1% of 10 cores                                                                │
│ CPU PEAK             |        100.0% of 10 cores                                                                │
│ RAM AVERAGE          | 8.63 GB / 24.0 GB (36.0%)                                                                │
│ RAM PEAK             | 9.96 GB / 24.0 GB (41.5%)                                                                │
│ GPU                  |             Not available                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Process - Summary ───────────────────────────────────────────────╮
│ TOTAL PROCESS SAMPLES |                               117                                                       │
│ CPU AVERAGE           | 118.2% (~1.2 cores of 10.0 cores)                                                       │
│ CPU PEAK              | 152.7% (~1.5 cores of 10.0 cores)                                                       │
│ RAM AVERAGE           |         2.56 GB / 24.0 GB (10.7%)                                                       │
│ RAM PEAK              |         4.87 GB / 24.0 GB (20.3%)                                                       │
│ GPU                   |                     Not available                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Model Layer - Summary ─────────────────────────────────────────────╮
│ TOTAL SAMPLES TAKEN                               |      1                                                      │
│ TOTAL MODELS SEEN                                 |      1                                                      │
│ AVERAGE MODEL MEMORY                              | 255 MB                                                      │
│ PEAK MODEL MEMORY                                 | 255 MB                                                      │
│ TOP-3 ACTIVATIONS                                 |                                                             │
│   • distilbert.transformer.layer.0.ffn.lin1         128 MB                                                      │
│   • distilbert.transformer.layer.0.ffn.activation   128 MB                                                      │
│   • distilbert.transformer.layer.1.ffn.lin1         128 MB                                                      │
│ TOP-3 GRADIENTS                                   |                                                             │
│   • distilbert.transformer.layer.5.ffn.activation   128 MB                                                      │
│   • distilbert.transformer.layer.5.ffn.lin1         128 MB                                                      │
│   • distilbert.transformer.layer.4.ffn.activation   128 MB                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Activation & Gradient - Summary ────────────────────────────────────────╮
│ Approx ACTIVATION AVG | 1.67 GB                                                                                 │
│ Approx ACTIVATION MAX | 3.10 GB                                                                                 │
│ Approx GRADIENT AVG   | 1.60 GB                                                                                 │
│ Approx GRADIENT MAX   | 3.16 GB                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
tracker.log_summaries()
